In [1]:
from lohrasb.best_estimator import BaseModel
from optuna.pruners import HyperbandPruner
from optuna.samplers._tpe.sampler import TPESampler
from sklearn.model_selection import KFold,train_test_split
import pandas as pd
import optuna
from sklearn.pipeline import Pipeline
from feature_engine.imputation import (
    CategoricalImputer,
    MeanMedianImputer
    )
from category_encoders import OrdinalEncoder
from sklearn.metrics import mean_absolute_error,r2_score
from interpret.glassbox import ExplainableBoostingRegressor
from interpret import show


2023-07-03 21:35:10,480 :: matplotlib :: matplotlib data path: /Users/hjavedani/Documents/Lohrasb/.venv/lib/python3.10/site-packages/matplotlib/mpl-data
2023-07-03 21:35:10,486 :: matplotlib :: CONFIGDIR=/Users/hjavedani/.matplotlib
2023-07-03 21:35:10,489 :: matplotlib :: interactive is False
2023-07-03 21:35:10,490 :: matplotlib :: platform is darwin
2023-07-03 21:35:10,506 :: graphviz._tools :: deprecate positional args: graphviz.backend.piping.pipe(['renderer', 'formatter', 'neato_no_op', 'quiet'])
2023-07-03 21:35:10,508 :: graphviz._tools :: deprecate positional args: graphviz.backend.rendering.render(['renderer', 'formatter', 'neato_no_op', 'quiet'])
2023-07-03 21:35:10,510 :: graphviz._tools :: deprecate positional args: graphviz.backend.unflattening.unflatten(['stagger', 'fanout', 'chain', 'encoding'])
2023-07-03 21:35:10,512 :: graphviz._tools :: deprecate positional args: graphviz.backend.viewing.view(['quiet'])
2023-07-03 21:35:10,516 :: graphviz._tools :: deprecate positio

#### Example 2 :Computer Hardware Data Set (a regression problem)
  
https://archive.ics.uci.edu/ml/datasets/Computer+Hardware

#### Part 1: Use BestModel in sklearn pipeline


In [2]:
urldata= "https://archive.ics.uci.edu/ml/machine-learning-databases/cpu-performance/machine.data"
# column names
col_names=[
    "vendor name",
    "Model Name",
    "MYCT",
    "MMIN",
    "MMAX",
    "CACH",
    "CHMIN",
    "CHMAX",
    "PRP"
]
# read data
data = pd.read_csv(urldata,header=None,names=col_names,sep=',')
data

,vendor name,Model Name,MYCT,MMIN,MMAX,CACH,CHMIN,CHMAX,PRP
adviser,32/60,125,256,6000,256,16,128,198,199
amdahl,470v/7,29,8000,32000,32,8,32,269,253
amdahl,470v/7a,29,8000,32000,32,8,32,220,253
amdahl,470v/7b,29,8000,32000,32,8,32,172,253
amdahl,470v/7c,29,8000,16000,32,8,16,132,132
...,...,...,...,...,...,...,...,...,...
sperry,80/8,124,1000,8000,0,1,8,42,37
sperry,90/80-model-3,98,1000,8000,32,2,8,46,50
sratus,32,125,2000,8000,0,2,14,52,41
wang,vs-100,480,512,8000,32,0,0,67,47


#### Train test split

In [3]:
X = data.loc[:, data.columns != "PRP"]
y = data.loc[:, data.columns == "PRP"]
y = y.values.ravel()


X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.33, random_state=42)


#### Find feature types for later use

In [4]:
int_cols =  X_train.select_dtypes(include=['int']).columns.tolist()
float_cols =  X_train.select_dtypes(include=['float']).columns.tolist()
cat_cols =  X_train.select_dtypes(include=['object']).columns.tolist()


####  Define estimator and set its arguments  

In [5]:
estimator = ExplainableBoostingRegressor()
estimator_params= {
                    "learning_rate":[0.01,0.1], 
                    "validation_size":[0.15,0.20],
                    "early_stopping_rounds":[50,200], 
                    "early_stopping_tolerance":[0.00001,0.001], 
                    "max_rounds":[5000,20000], 
                    "min_samples_leaf":[2,5], 
                    "max_leaves":[2,5]  
                      }

kwargs = {  # params for fit method  
            'fit_optuna_kwargs' :{
            'sample_weight':None,
            },
            # params for OptunaSearch
            'main_optuna_kwargs' : {
            'estimator':estimator,
            'estimator_params':estimator_params,
            'refit':True,
            'measure_of_accuracy':"mean_absolute_error(y_true, y_pred, multioutput='uniform_average')",

            },
            'train_test_split_kwargs':{
                'test_size':.3,
                            
            },
            'study_search_kwargs':{
                'storage':None,
                'sampler':TPESampler(),
                'pruner':HyperbandPruner(),
                'study_name':"example of optuna optimizer",
                'direction':"maximize",
                'load_if_exists':False,
            },
            'optimize_kwargs':{
                # optuna optimization params
                'n_trials':20,
                'timeout':600,
                'catch':(),
                'callbacks':None,
                'gc_after_trial':False,
                'show_progress_bar':False,
            }
}



In [6]:
obj = BaseModel().optimize_by_optuna(
    kwargs=kwargs
        )

#### Build sklearn pipeline

In [7]:


pipeline =Pipeline([
            # int missing values imputers
            ('intimputer', MeanMedianImputer(
                imputation_method='median', variables=int_cols)),
            # category missing values imputers
            ('catimputer', CategoricalImputer(variables=cat_cols)),
            #
            ('catencoder', OrdinalEncoder()),
            # regression model 
            ('obj', obj),


 ])



#### Run Pipeline

In [8]:
pipeline.fit(X_train,y_train)
y_pred = pipeline.predict(X_test)


[I 2023-07-03 21:35:11,740] A new study created in memory with name: example of optuna optimizer


2023-07-03 21:35:11,744 :: interpret.utils._native :: EBM lib loading.
2023-07-03 21:35:11,745 :: interpret.utils._native :: Loading native on Darwin | debug = False
2023-07-03 21:35:11,791 :: interpret.utils._binning :: Creating native dataset
2023-07-03 21:35:16,088 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:16,093 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:16,096 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:16,098 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:16,100 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:16,102 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:16,105 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:16,108 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:16,112 :: interpret.utils._binning :: Creating native dataset
2023-07-03 21:35:16,117 :: interpret.glassbox.ebm.ebm :: Estimating with FAST
2023-07-03 21:35:16,314 :: interpret.glassbox.ebm.bin 

[I 2023-07-03 21:35:16,330] Trial 0 finished with value: 16.87398949462853 and parameters: {'learning_rate': 0.06947248706611646, 'validation_size': 0.17512465805463795, 'early_stopping_rounds': 127, 'early_stopping_tolerance': 0.0003001707041208225, 'max_rounds': 13248, 'min_samples_leaf': 4, 'max_leaves': 5}. Best is trial 0 with value: 16.87398949462853.


2023-07-03 21:35:16,341 :: interpret.utils._binning :: Creating native dataset
2023-07-03 21:35:17,167 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:17,169 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:17,172 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:17,173 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:17,176 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:17,178 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:17,180 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:17,182 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:17,185 :: interpret.utils._binning :: Creating native dataset
2023-07-03 21:35:17,189 :: interpret.glassbox.ebm.ebm :: Estimating with FAST
2023-07-03 21:35:17,931 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:17,946 :: interpret.glassbox.ebm.bin :: eval_terms


[I 2023-07-03 21:35:17,949] Trial 1 finished with value: 17.276630235432123 and parameters: {'learning_rate': 0.05376002621321309, 'validation_size': 0.16055152072309012, 'early_stopping_rounds': 149, 'early_stopping_tolerance': 0.0004421058711751661, 'max_rounds': 8791, 'min_samples_leaf': 4, 'max_leaves': 5}. Best is trial 1 with value: 17.276630235432123.


...

2023-07-03 21:35:41,579 :: interpret.utils._binning :: Creating native dataset
2023-07-03 21:35:42,546 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:42,547 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:42,549 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:42,551 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:42,553 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:42,556 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:42,558 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:42,560 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:42,563 :: interpret.utils._binning :: Creating native dataset
2023-07-03 21:35:42,566 :: interpret.glassbox.ebm.ebm :: Estimating with FAST
2023-07-03 21:35:44,186 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:44,195 :: interpret.glassbox.ebm.bin :: eval_terms


[I 2023-07-03 21:35:44,198] Trial 18 finished with value: 13.671903149689467 and parameters: {'learning_rate': 0.026405965615698416, 'validation_size': 0.1999202717864676, 'early_stopping_rounds': 192, 'early_stopping_tolerance': 0.00016376703543045706, 'max_rounds': 6340, 'min_samples_leaf': 4, 'max_leaves': 2}. Best is trial 17 with value: 20.696284939063283.


2023-07-03 21:35:44,239 :: interpret.utils._binning :: Creating native dataset
2023-07-03 21:35:45,619 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:45,622 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:45,624 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:45,626 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:45,628 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:45,631 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:45,633 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:45,636 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:45,639 :: interpret.utils._binning :: Creating native dataset
2023-07-03 21:35:45,642 :: interpret.glassbox.ebm.ebm :: Estimating with FAST
2023-07-03 21:35:50,538 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:50,546 :: interpret.glassbox.ebm.bin :: eval_terms


[I 2023-07-03 21:35:50,549] Trial 19 finished with value: 20.47509722043161 and parameters: {'learning_rate': 0.010513076746714535, 'validation_size': 0.1995127485788442, 'early_stopping_rounds': 167, 'early_stopping_tolerance': 2.2385146335644838e-05, 'max_rounds': 9790, 'min_samples_leaf': 5, 'max_leaves': 2}. Best is trial 17 with value: 20.696284939063283.


2023-07-03 21:35:50,550 :: dev :: If refit is set to True, the optimal model will be refit on the entire dataset, i.e., X_train and y_train!
2023-07-03 21:35:50,550 :: dev :: If refit is set to True, the optimal model will be refit on the entire dataset, i.e., X_train and y_train!
2023-07-03 21:35:50,560 :: interpret.utils._binning :: Creating native dataset
2023-07-03 21:35:51,088 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:51,090 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:51,092 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:51,094 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:51,096 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:51,097 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:51,100 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:51,103 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:51,105 :: interpret.utils._binning :: Creating native dataset
2023-07-03 21:35

#### Check performance of the pipeline

In [9]:
print('r2 score : ')
print(r2_score(y_test,y_pred))


r2 score : 
0.57528751857482


#### Part 2:  Use BestModel as a standalone estimator 

In [10]:
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.33, random_state=42)


#### Transform features to make them ready for model input

In [11]:
transform_pipeline =Pipeline([
            # int missing values imputers
            ('intimputer', MeanMedianImputer(
                imputation_method='median', variables=int_cols)),
            # category missing values imputers
            ('catimputer', CategoricalImputer(variables=cat_cols)),
            #
            ('catencoder', OrdinalEncoder()),
            # classification model

 ])

#### Transform X_train and X_test

In [12]:
X_train=transform_pipeline.fit_transform(X_train,y_train)
X_test=transform_pipeline.transform(X_test)


#### Train model and predict

In [13]:
obj.fit(X_train,y_train)
y_pred = obj.predict(X_test)

[I 2023-07-03 21:35:51,398] A new study created in memory with name: example of optuna optimizer


2023-07-03 21:35:51,410 :: interpret.utils._binning :: Creating native dataset
2023-07-03 21:35:51,642 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:51,647 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:51,660 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:51,662 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:51,665 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:51,667 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:51,669 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:51,684 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:51,689 :: interpret.utils._binning :: Creating native dataset
2023-07-03 21:35:51,692 :: interpret.glassbox.ebm.ebm :: Estimating with FAST
2023-07-03 21:35:51,832 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:51,841 :: interpret.glassbox.ebm.bin :: eval_terms


[I 2023-07-03 21:35:51,844] Trial 0 finished with value: 13.08859825989703 and parameters: {'learning_rate': 0.06989575212117201, 'validation_size': 0.17191775101986767, 'early_stopping_rounds': 81, 'early_stopping_tolerance': 0.0003896138513525824, 'max_rounds': 19315, 'min_samples_leaf': 3, 'max_leaves': 2}. Best is trial 0 with value: 13.08859825989703.


2023-07-03 21:35:51,854 :: interpret.utils._binning :: Creating native dataset
2023-07-03 21:35:51,973 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:51,975 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:51,978 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:51,981 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:51,984 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:51,986 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:51,988 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:51,990 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:51,992 :: interpret.utils._binning :: Creating native dataset
2023-07-03 21:35:51,995 :: interpret.glassbox.ebm.ebm :: Estimating with FAST
2023-07-03 21:35:52,193 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:35:52,202 :: interpret.glassbox.ebm.bin :: eval_terms


[I 2023-07-03 21:35:52,205] Trial 1 finished with value: 12.381911928292059 and parameters: {'learning_rate': 0.024565060027954326, 'validation_size': 0.1779018044761367, 'early_stopping_rounds': 171, 'early_stopping_tolerance': 0.0003193778222117331, 'max_rounds': 14811, 'min_samples_leaf': 5, 'max_leaves': 4}. Best is trial 0 with value: 13.08859825989703.


...

2023-07-03 21:36:00,316 :: interpret.utils._binning :: Creating native dataset
2023-07-03 21:36:00,396 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:36:00,399 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:36:00,400 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:36:00,403 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:36:00,405 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:36:00,407 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:36:00,409 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:36:00,412 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:36:00,414 :: interpret.utils._binning :: Creating native dataset
2023-07-03 21:36:00,416 :: interpret.glassbox.ebm.ebm :: Estimating with FAST
2023-07-03 21:36:00,680 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:36:00,688 :: interpret.glassbox.ebm.bin :: eval_terms


[I 2023-07-03 21:36:00,690] Trial 18 finished with value: 13.22382294426842 and parameters: {'learning_rate': 0.061773153574108564, 'validation_size': 0.15785170664909687, 'early_stopping_rounds': 153, 'early_stopping_tolerance': 0.0008447535870680983, 'max_rounds': 9814, 'min_samples_leaf': 4, 'max_leaves': 5}. Best is trial 14 with value: 16.050314195986658.


2023-07-03 21:36:00,731 :: interpret.utils._binning :: Creating native dataset
2023-07-03 21:36:00,797 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:36:00,800 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:36:00,802 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:36:00,804 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:36:00,806 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:36:00,808 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:36:00,811 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:36:00,813 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:36:00,816 :: interpret.utils._binning :: Creating native dataset
2023-07-03 21:36:00,819 :: interpret.glassbox.ebm.ebm :: Estimating with FAST
2023-07-03 21:36:01,377 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:36:01,386 :: interpret.glassbox.ebm.bin :: eval_terms


[I 2023-07-03 21:36:01,389] Trial 19 finished with value: 12.70857853023344 and parameters: {'learning_rate': 0.04283437003756546, 'validation_size': 0.17003770752283, 'early_stopping_rounds': 119, 'early_stopping_tolerance': 0.0008674158949866797, 'max_rounds': 7267, 'min_samples_leaf': 3, 'max_leaves': 4}. Best is trial 14 with value: 16.050314195986658.


2023-07-03 21:36:01,389 :: dev :: If refit is set to True, the optimal model will be refit on the entire dataset, i.e., X_train and y_train!
2023-07-03 21:36:01,389 :: dev :: If refit is set to True, the optimal model will be refit on the entire dataset, i.e., X_train and y_train!
2023-07-03 21:36:01,398 :: interpret.utils._binning :: Creating native dataset
2023-07-03 21:36:01,629 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:36:01,631 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:36:01,633 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:36:01,635 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:36:01,637 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:36:01,639 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:36:01,641 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:36:01,643 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:36:01,645 :: interpret.utils._binning :: Creating native dataset
2023-07-03 21:36

#### Check performance of the model

In [14]:
print('r2 score : ')
print(r2_score(y_test,y_pred))
print('mean_absolute_error : ')
print(mean_absolute_error(y_test,y_pred))


r2 score : 
0.5740959702103923
mean_absolute_error : 
42.83460423667753


In [15]:
ebm=obj.get_best_estimator()
print(ebm)

ExplainableBoostingRegressor(early_stopping_rounds=124,
                             early_stopping_tolerance=0.0008269484382500892,
                             learning_rate=0.042870332279302696, max_leaves=5,
                             max_rounds=7052, min_samples_leaf=3,
                             validation_size=0.159926048317232)


In [16]:
obj.best_estimator

ExplainableBoostingRegressor(early_stopping_rounds=124,
                             early_stopping_tolerance=0.0008269484382500892,
                             learning_rate=0.042870332279302696, max_leaves=5,
                             max_rounds=7052, min_samples_leaf=3,
                             validation_size=0.159926048317232)

#### Get fitted randomized search object and its attributes

In [17]:
OptunaObj = obj.get_optimized_object()
OptunaObj

FrozenTrial(number=14, state=TrialState.COMPLETE, values=[16.050314195986658], datetime_start=datetime.datetime(2023, 7, 3, 21, 35, 58, 593308), datetime_complete=datetime.datetime(2023, 7, 3, 21, 35, 59, 7023), params={'learning_rate': 0.042870332279302696, 'validation_size': 0.159926048317232, 'early_stopping_rounds': 124, 'early_stopping_tolerance': 0.0008269484382500892, 'max_rounds': 7052, 'min_samples_leaf': 3, 'max_leaves': 5}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=0.1, log=False, low=0.01, step=None), 'validation_size': FloatDistribution(high=0.2, log=False, low=0.15, step=None), 'early_stopping_rounds': IntDistribution(high=200, log=False, low=50, step=1), 'early_stopping_tolerance': FloatDistribution(high=0.001, log=False, low=1e-05, step=None), 'max_rounds': IntDistribution(high=20000, log=False, low=5000, step=1), 'min_samples_leaf': IntDistribution(high=5, log=False, low=2, step=1), 'max_leaves': IntD

#### This visualizations provided is for global explanations.



In [18]:
ebm_global = ebm.explain_global()
show(ebm_global)


2023-07-03 21:36:01,882 :: interpret.provider.visualize :: Detected non-cloud environment.
2023-07-03 21:36:02,533 :: interpret.visual.udash :: Generating mini dash
2023-07-03 21:36:02,548 :: interpret.visual.udash :: Generated mini dash


<!-- http://127.0.0.1:7001/5311248368/ -->

#### This visualizations provided is for local explanations.


In [ ]:
ebm_local = ebm.explain_local(X_test[:5], y_test[:5])
show(ebm_local)


2023-07-03 21:36:02,561 :: interpret.utils._binning :: Unifying data
2023-07-03 21:36:02,564 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:36:02,713 :: interpret.glassbox.ebm.bin :: eval_terms
2023-07-03 21:36:02,756 :: interpret.visual.udash :: Generating mini dash
2023-07-03 21:36:02,784 :: interpret.visual.udash :: Generated mini dash


<!-- http://127.0.0.1:7001/5319473904/ -->

2023-07-03 21:36:03,331 :: matplotlib :: CACHEDIR=/Users/hjavedani/.matplotlib
2023-07-03 21:36:03,336 :: matplotlib.font_manager :: Using fontManager instance from /Users/hjavedani/.matplotlib/fontlist-v330.json
2023-07-03 21:36:07,966 :: interpret.visual.udash :: No overall plot to display: -1|ExplainableBoostingRegressor_1
